In [1]:
!wget https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam -O example.bam
!wget https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam.bai -O example.bam.bai
!touch example.bam.bai # ensures no errors from pysam

--2023-07-15 10:21:35--  https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam
Resolving aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)... 52.216.32.33, 52.216.61.225, 3.5.10.213, ...
Connecting to aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)|52.216.32.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4993966 (4.8M) [binary/octet-stream]
Saving to: ‘example.bam’

example.bam         100%[===================>]   4.76M  6.67MB/s    in 0.7s    

2023-07-15 10:21:36 (6.67 MB/s) - ‘example.bam’ saved [4993966/4993966]

--2023-07-15 10:21:36--  https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam.bai
Resolving aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)... 52.216.32.33, 52.216.61.225, 3.5.10.213, ...
Connecting to aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)|52.216.32.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28176 (28K) [binary/octet-stream]
Saving to: ‘example.bam.bai’

example.bam.bai     100%[====

In [1]:
import pathlib
import io

import pysam
import polars as pl
import pyarrow as pa
import pandas as pd
import bioframe

import dask_ngs



In [2]:
from io import BytesIO

import bioframe
import dask
import dask.dataframe as dd
import oxbow as ox
import pandas as pd
import pyarrow.ipc


def _read_bam_query_from_path(
    path: str, chrom: str, start: int, end: int
) -> pd.DataFrame:
    stream = BytesIO(ox.read_bam(path, f"{chrom}:{start}-{end}"))
    ipc = pyarrow.ipc.open_file(stream)
    return ipc.read_pandas()

In [47]:
from gzip import GzipFile
from io import BytesIO

import numpy as np
import pandas as pd

BAI_MIN_SHIFT = 14
BAI_DEPTH = 5
COMPRESSED_POSITION_SHIFT = 16
UNCOMPRESSED_POSITION_MASK = 0xffff
BLOCKSIZE = 65536

def read_bai(path):
    """
    https://samtools.github.io/hts-specs/SAMv1.pdf
    """
    int_kwargs = {'byteorder': 'little', 'signed': False}
    f = open(path, "rb")
    # read the 4-byte magic number
    magic = f.read(4)

    # read the number of reference sequences
    n_ref = int.from_bytes(f.read(4), **int_kwargs)

    # read the reference sequence indices
    references = []
    for i in range(n_ref):
        ref = {'ref_id': i}

        # The "Bin Index"
        chunks = []
        n_bin = int.from_bytes(f.read(4), **int_kwargs)
        for _ in range(n_bin):
            # bin number
            bin_id = int.from_bytes(f.read(4), **int_kwargs)

            if bin_id == 37450:
                # This is an entry that describes the "pseudo-bin" for the 
                # reference, using the same byte layout as normal bins but 
                # interpreted differently.
                # The ref beg/ref end fields locate the first and last reads on
                # this reference sequence, whether they are mapped or placed 
                # unmapped. Thus they are equal to the minimum chunk beg and 
                # maximum chunk end respectively.
                n_chunk = int.from_bytes(f.read(4), **int_kwargs)  # always 2
                # Not really a chunk
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                ref["ref_beg.cpos"] = vpos >> COMPRESSED_POSITION_SHIFT
                ref["ref_beg.upos"] = vpos & UNCOMPRESSED_POSITION_MASK
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                ref["ref_end.cpos"] = vpos >> COMPRESSED_POSITION_SHIFT
                ref["ref_end.upos"] = vpos & UNCOMPRESSED_POSITION_MASK
                # Not really a chunk
                ref["n_mapped"] = int.from_bytes(f.read(8), **int_kwargs)
                ref["n_unmapped"] = int.from_bytes(f.read(8), **int_kwargs)
                continue

            n_chunk = int.from_bytes(f.read(4), **int_kwargs)
            for _ in range(n_chunk):
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                chunk_beg_cpos = vpos >> COMPRESSED_POSITION_SHIFT
                chunk_beg_upos = vpos & UNCOMPRESSED_POSITION_MASK
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                chunk_end_cpos = vpos >> COMPRESSED_POSITION_SHIFT
                chunk_end_upos = vpos & UNCOMPRESSED_POSITION_MASK

                chunks.append((bin_id, chunk_beg_cpos, chunk_beg_upos, chunk_end_cpos, chunk_end_upos))   

            ref["bins"] = chunks

        # The "Linear Index"
        ref["ioffsets"] = []
        n_intv = int.from_bytes(f.read(4), **int_kwargs)
        for _ in range(n_intv):
            vpos = int.from_bytes(f.read(8), **int_kwargs)
            ioffset_cpos = vpos >> COMPRESSED_POSITION_SHIFT
            ioffset_upos = vpos & UNCOMPRESSED_POSITION_MASK
            ref["ioffsets"].append((ioffset_cpos, ioffset_upos))

        references.append(ref)

    return references       


In [48]:
   
# Convert the deconstructed virtual positions into dataframes
def make_dataframe(references):

    for ref in references:
        if not 'bins' in ref:
            continue

        ref["bins"] = pd.DataFrame(
            ref["bins"],
            columns=["bin_id", "chunk_beg.cpos", "chunk_beg.upos", "chunk_end.cpos", "chunk_end.upos"]
        )
        ref["ioffsets"] = pd.DataFrame(
            ref["ioffsets"],
            columns=["ioffset.cpos", "ioffset.upos"]
        )

    # Check for the optional n_no_coor at the end of the file
    try:
        n_no_coor = int.from_bytes(f.read(8), **int_kwargs)
    except:
        n_no_coor = None


    return references, n_no_coor

bai = read_bai('example.bam.bai')
ref = bai[0]
ref['bins']
df = make_dataframe(bai)
df

([{'ref_id': 0,
   'bins':     bin_id  chunk_beg.cpos  chunk_beg.upos  chunk_end.cpos  chunk_end.upos
   0      585          157643           61968          163802            9643
   1      585          456717           19251          456717           20088
   2       73          720450           14227          720450           27457
   3       73         1825729           40851         1832074             276
   4       73         3306112             550         3306112           11849
   5       73         4929731           62149         4937687           11573
   6      586          941494           61277          941494           64577
   7      586         1098225           39987         1098225           45238
   8      586         1273813           11833         1273813           18402
   9      586         1633523           37951         1633523           42945
   10     587         2239862           10042         2239862           22158
   11     587         2496504           

In [97]:
offsets = ref['ioffsets']['ioffset.cpos']

#o = offsets[:, 1:,] - offsets[:,:-1]
diffs = [0] + [y - x for x, y in zip(offsets, offsets[1:])]
#ref['ioffsets'].append(diffs)  
ref['ioffsets']['ioffset.cpos.diff'] = diffs
offsets = ref['ioffsets']
offsets

,ioffset.cpos,ioffset.upos,ioffset.cpos.diff,chunk_id
0,38660,0,0,0
1,38660,0,0,0
2,38660,0,0,0
3,38660,0,0,0
4,38660,0,0,0
5,38660,0,0,0
6,157643,61968,118983,0
7,456717,19251,299074,0
8,720450,14227,263733,0
9,941494,61277,221044,0


Write a function that creates a list of consolidated chunks each having a size no greater than an input (chunk size)

Outputs a list of start position and end position. Keep the uncompressed and compressed information in the function.

In [102]:
# TODO: Make this more Pythonic

def get_chunk_info(offsets, chunksize: int):
    index = 0
    sum = 0
    for i in range(0, len(offsets)):
        sum += offsets.iloc[i,2] # 2 refers to the diff column
        if sum > chunksize:
            index += 1
            sum = 0
        offsets.at[i,'chunk_id'] = index
    return offsets

o = get_chunk_info(offsets, 1_000_000)
o

,ioffset.cpos,ioffset.upos,ioffset.cpos.diff,chunk_id
0,38660,0,0,0
1,38660,0,0,0
2,38660,0,0,0
3,38660,0,0,0
4,38660,0,0,0
5,38660,0,0,0
6,157643,61968,118983,0
7,456717,19251,299074,0
8,720450,14227,263733,0
9,941494,61277,221044,0


In [3]:
chromsizes = bioframe.fetch_chromsizes("hg38")
chunksize = 10_000_000
chunk_spans = bioframe.binnify(chromsizes, chunksize)
chunk_spans["start"] += 1
chunk_spans

,chrom,start,end
0,chr1,1,10000000
1,chr1,10000001,20000000
2,chr1,20000001,30000000
3,chr1,30000001,40000000
4,chr1,40000001,50000000
...,...,...,...
318,chrY,20000001,30000000
319,chrY,30000001,40000000
320,chrY,40000001,50000000
321,chrY,50000001,57227415


In [50]:
path = "example.bam"
chunks = [
    dask.delayed(_read_bam_query_from_path)(path, chrom, start, end)
    for chrom, start, end in chunk_spans.to_numpy()
]
df = dd.from_delayed(chunks)
df

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual,end
npartitions=323,,,,,,,,,,,,
,object,uint16,category[known],int32,uint8,object,category[known],int32,int32,object,object,int32
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
df.partitions[0].compute()

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual,end
0,SRR4435251::::313654063,83,chr1,82736,0,100M,chr1,82517,-319,TAAAAAAGAATGCAGATATTACAAAACCAGTTTACAAAAGTTACTA...,??????????????????????????????????????????????...,82835
1,SRR4435251::::1356039,147,chr1,82742,0,100M,chr1,82511,-331,AGAATGCAGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAA...,??????????????????????????????????????????????...,82841
2,SRR4435251::::270890793,163,chr1,82744,0,100M,chr1,82982,338,AATGCAGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAATA...,??????????????????????????????????????????????...,82843
3,SRR4435251::::1356040,83,chr1,82748,0,100M,chr1,82488,-360,CAGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAATAAAAA...,??????????????????????????????????????????????...,82847
4,SRR4435251::::1482538,147,chr1,82749,0,100M,chr1,82506,-343,AGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAATAAAAAC...,??????????????????????????????????????????????...,82848
...,...,...,...,...,...,...,...,...,...,...,...,...
160173,SRR4435251::::1378461,147,chr1,528790,0,100M,chr1,528599,-291,CTTGTCTGGTCTGGCTCTGCCCCACTCTCCTTCTCACCTAGTTGGA...,??????????????????????????????????????????????...,528889
160174,SRR4435251::::455831805,83,chr1,528792,0,100M,chr1,528483,-409,TGTCTGGTCTGGCTCTGCCCCACTCTCCTTCTCACCTAGTTGGAAT...,??????????????????????????????????????????????...,528891
160175,SRR4435251::::1378464,83,chr1,528800,0,100M,chr1,528695,-205,CTGGCTCTGCCCCACTCTCCTTCTCTCCTAGTTGGAATCCCTAGCT...,??5????????5?5???5'55+???+55????5??5??55????5?...,528899
160176,SRR4435251::::1378465,147,chr1,528803,0,100M,chr1,528567,-336,GCTCTGCCCCACTCTCCTTCTCACCTAGTTGGAATCCCTAGCTACT...,????????5?5???????????????????????????????????...,528902


In [3]:
file = pathlib.Path("example.bam")
region = ("chr1", 1, 8_000_000)

In [4]:
%timeit pysam_run(file, *region)

2.08 s ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit oxbow_polars(file, *region)

1.2 s ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit oxbow_pandas(file, *region)

1.24 s ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
